In [2]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import regexp_tokenize
from nltk.corpus import movie_reviews
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Definimos el set de stopwords
english_sw = set(stopwords.words("english"))

- **Ejercicio 1:** Utilizando una representación de bolsa de palabras (BoW) se quiere que los rasgos del
vocabulario no sean palabras, sino n-gramas. Los n-gramas son secuencias contiguas de n elementos en un texto, cruciales para captar la información contextual. Pueden mejorar la representación de los textos al no considerar palabras sueltas, sino combinaciones de estas, lo que proporciona un contexto más rico.  
Dadas las siguientes frases: “Estamos ya a finales de febrero.”, “En febrero sigue haciendo frío.”, “Esto es una frase de ejemplo que habla de un mes del año.” Se pide lo siguiente:
- Generar una representación BoW con bigramas (2-grams).
- Generar una representación BoW con unigramas, bigramas y trigramas.
En cada caso se debe de imprimir el vocabulario generado y la matriz de rasgosdocumentos.

In [3]:
corpus = [
    "Estamos ya a finales de febrero.",
    "En febrero sigue haciendo frío.",
    "Esto es una frase de ejemplo que habla de un mes del año."
]

# Representación BoW con bigramas
vectorizer1 = CountVectorizer(ngram_range=(2, 2))
bow_encoded1 = vectorizer1.fit_transform(corpus)
print("Repre. con BoW de bigramas")
print("Vocabulario:")
print(vectorizer1.get_feature_names_out())
print("Matriz rasgos-documentos:")
print(bow_encoded1.toarray())

# Representación BoW con unigramas, bigramas y trigramas
vectorizer2 = CountVectorizer(ngram_range=(1, 3))
bow_encoded2 = vectorizer2.fit_transform(corpus)
print("\n\nRepre. con BoW de unigramas, bigramas y trigramas:")
print("Vocabulario:")
print(vectorizer2.get_feature_names_out())
print("Matriz rasgos-documentos:")
print(bow_encoded2.toarray())

Repre. con BoW de bigramas
Vocabulario:
['de ejemplo' 'de febrero' 'de un' 'del año' 'ejemplo que' 'en febrero'
 'es una' 'estamos ya' 'esto es' 'febrero sigue' 'finales de' 'frase de'
 'habla de' 'haciendo frío' 'mes del' 'que habla' 'sigue haciendo'
 'un mes' 'una frase' 'ya finales']
Matriz rasgos-documentos:
[[0 1 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 1 0 0 0 1 0 0 0 1 0 0 1 0 0 0]
 [1 0 1 1 1 0 1 0 1 0 0 1 1 0 1 1 0 1 1 0]]


Repre. con BoW de unigramas, bigramas y trigramas:
Vocabulario:
['año' 'de' 'de ejemplo' 'de ejemplo que' 'de febrero' 'de un' 'de un mes'
 'del' 'del año' 'ejemplo' 'ejemplo que' 'ejemplo que habla' 'en'
 'en febrero' 'en febrero sigue' 'es' 'es una' 'es una frase' 'estamos'
 'estamos ya' 'estamos ya finales' 'esto' 'esto es' 'esto es una'
 'febrero' 'febrero sigue' 'febrero sigue haciendo' 'finales' 'finales de'
 'finales de febrero' 'frase' 'frase de' 'frase de ejemplo' 'frío' 'habla'
 'habla de' 'habla de un' 'haciendo' 'haciendo frío' 'mes' 'm

---
- **Ejercicio 2:** Dadas varias frases, generar una representación BoW con función de pesado TF-IDF. Imprimir el vocabulario generado y la matriz de rasgos-documentos.

In [4]:
# Utilizamos el corpus anterior
vectorizer = TfidfVectorizer()
bow_encoded = vectorizer.fit_transform(corpus)
print("Vocabulario:")
print(vectorizer.get_feature_names_out())
print("Matriz rasgos-documentos:")
print(bow_encoded.toarray())

Vocabulario:
['año' 'de' 'del' 'ejemplo' 'en' 'es' 'estamos' 'esto' 'febrero' 'finales'
 'frase' 'frío' 'habla' 'haciendo' 'mes' 'que' 'sigue' 'un' 'una' 'ya']
Matriz rasgos-documentos:
[[0.         0.37302199 0.         0.         0.         0.
  0.49047908 0.         0.37302199 0.49047908 0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.49047908]
 [0.         0.         0.         0.         0.46735098 0.
  0.         0.         0.35543247 0.         0.         0.46735098
  0.         0.46735098 0.         0.         0.46735098 0.
  0.         0.        ]
 [0.27406418 0.41686575 0.27406418 0.27406418 0.         0.27406418
  0.         0.27406418 0.         0.         0.27406418 0.
  0.27406418 0.         0.27406418 0.27406418 0.         0.27406418
  0.27406418 0.        ]]


---
- **Ejercicio 3:** Dadas las siguientes frases:
    - "El saque de Nadal es imparable.",
    - "Me encanta jugar al tenis los fines de semana.",
    - "¿Viste el último partido de Federer?",
    - "Necesito una nueva raqueta de tenis.",
    - "El torneo de Wimbledon es mi favorito.",
    - "Practicar el revés es fundamental para mejorar.",
    - "El tenis es un deporte que requiere mucha concentración.",
    - "¿Quién es tu tenista favorito?",
    - "La pista de tierra batida es muy exigente.",
    - "Vamos a jugar un partido de dobles."  

Se pide lo siguiente:
- Generar sus unigramas y mostrar el vocabulario con la frecuencia de cada uno en toda la colección.
- Utilizar pandas para mostrar los unigramas por documento (por frase).

In [5]:
corpus = [
    "El saque de Nadal es imparable.",
    "Me encanta jugar al tenis los fines de semana.",
    "¿Viste el último partido de Federer?",
    "Necesito una nueva raqueta de tenis.",
    "El torneo de Wimbledon es mi favorito.",
    "Practicar el revés es fundamental para mejorar.",
    "El tenis es un deporte que requiere mucha concentración.",
    "¿Quién es tu tenista favorito?",
    "La pista de tierra batida es muy exigente.",
    "Vamos a jugar un partido de dobles."  
]

# Mostramos el vocabulario con la frecuencia de cada unigrama
vectorizer = CountVectorizer()
bow_encoded = vectorizer.fit_transform(corpus)

for idx, unigram in enumerate(vectorizer.get_feature_names_out()):
    print(f"Unigrama: {unigram}; Frecuencia: {sum(bow_encoded.toarray()[:, idx])}")

# Mostramos los unigramas por documento
df = pd.DataFrame(bow_encoded.toarray(), columns=vectorizer.get_feature_names_out())
df.head()

Unigrama: al; Frecuencia: 1
Unigrama: batida; Frecuencia: 1
Unigrama: concentración; Frecuencia: 1
Unigrama: de; Frecuencia: 7
Unigrama: deporte; Frecuencia: 1
Unigrama: dobles; Frecuencia: 1
Unigrama: el; Frecuencia: 5
Unigrama: encanta; Frecuencia: 1
Unigrama: es; Frecuencia: 6
Unigrama: exigente; Frecuencia: 1
Unigrama: favorito; Frecuencia: 2
Unigrama: federer; Frecuencia: 1
Unigrama: fines; Frecuencia: 1
Unigrama: fundamental; Frecuencia: 1
Unigrama: imparable; Frecuencia: 1
Unigrama: jugar; Frecuencia: 2
Unigrama: la; Frecuencia: 1
Unigrama: los; Frecuencia: 1
Unigrama: me; Frecuencia: 1
Unigrama: mejorar; Frecuencia: 1
Unigrama: mi; Frecuencia: 1
Unigrama: mucha; Frecuencia: 1
Unigrama: muy; Frecuencia: 1
Unigrama: nadal; Frecuencia: 1
Unigrama: necesito; Frecuencia: 1
Unigrama: nueva; Frecuencia: 1
Unigrama: para; Frecuencia: 1
Unigrama: partido; Frecuencia: 2
Unigrama: pista; Frecuencia: 1
Unigrama: practicar; Frecuencia: 1
Unigrama: que; Frecuencia: 1
Unigrama: quién; Frecuen

,al,batida,concentración,de,deporte,dobles,el,encanta,es,exigente,...,tenista,tierra,torneo,tu,un,una,vamos,viste,wimbledon,último
0,0,0,0,1,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,1
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,0,0,0,1,0,0,1,0,1,0,...,0,0,1,0,0,0,0,0,1,0


---
- **Ejercicio 4:** Dada una frase cualquiera, utilizando la librería NLTK, se pide los siguiente:
    - Calcular y mostrar los bigramas
    - Calcular y mostrar los trigramas.
    - Haz lo mismo que en los dos casos anteriores, pero añadiendo relleno (inicio y fin de frase).  
    
El padding o relleno sirve para que cada componente del n-grama aparezca en todas las posiciones del n-grama.

In [6]:
sent = "Hoy es un día soleado por fin."
word_tokenized = word_tokenize(sent)

# Calculamos y mostramos los bigramas
bigrams = list(nltk.ngrams(word_tokenized, 2))
print(f"Bigramas: {bigrams}")

# Calculamos y mostramos los trigramas
trigrams = list(nltk.ngrams(word_tokenized, 3))
print(f"Trigramas: {trigrams}")

# Añadimos relleno
bigrams_pad = list(nltk.ngrams(word_tokenized, 2, pad_right=True, pad_left=True))
print(f"Bigramas con relleno: {bigrams_pad}")
trigrams_pad = list(nltk.ngrams(word_tokenized, 3, pad_right=True, pad_left=True))
print(f"Trigramas con relleno: {trigrams_pad}")

Bigramas: [('Hoy', 'es'), ('es', 'un'), ('un', 'día'), ('día', 'soleado'), ('soleado', 'por'), ('por', 'fin'), ('fin', '.')]
Trigramas: [('Hoy', 'es', 'un'), ('es', 'un', 'día'), ('un', 'día', 'soleado'), ('día', 'soleado', 'por'), ('soleado', 'por', 'fin'), ('por', 'fin', '.')]
Bigramas con relleno: [(None, 'Hoy'), ('Hoy', 'es'), ('es', 'un'), ('un', 'día'), ('día', 'soleado'), ('soleado', 'por'), ('por', 'fin'), ('fin', '.'), ('.', None)]
Trigramas con relleno: [(None, None, 'Hoy'), (None, 'Hoy', 'es'), ('Hoy', 'es', 'un'), ('es', 'un', 'día'), ('un', 'día', 'soleado'), ('día', 'soleado', 'por'), ('soleado', 'por', 'fin'), ('por', 'fin', '.'), ('fin', '.', None), ('.', None, None)]


---
- **Ejercicio 5:** Dadas las frases del ejercicio 3, se pide, utilizando la librería sklearn, obtener la matriz de similitud coseno.
    - Deberás utilizar la función de pesado binaria.
    - Deberás utilizar la función de pesado TF.
    - Deberás utilizar la función de pesado TF-IDF.

In [7]:
# Utilizando la función de pesado binaria
vectorizer_binary = CountVectorizer(binary=True)
bow_encoded_binary = vectorizer.fit_transform(corpus)
cosine_sim_binary = cosine_similarity(bow_encoded_binary.toarray())

# Utilizando la función de pesado TF
vectorizer_tf = CountVectorizer()
bow_encoded_tf = vectorizer_tf.fit_transform(corpus)
cosine_sim_tf = cosine_similarity(bow_encoded_tf.toarray())

# Utilizando la función de pesado TF-IDF
vectorizer_tfidf = TfidfVectorizer()
bow_encoded_tfidf = vectorizer_tfidf.fit_transform(corpus)
cosine_sim_tfidf = cosine_similarity(bow_encoded_tfidf.toarray())

print("Con fun. de pesado binaria:")
print(cosine_sim_binary)
print("\n\nCon fun. de pesado TF:")
print(cosine_sim_tf)
print("\n\nCon fun. de pesado TF-IDF:")
print(cosine_sim_tfidf)

Con fun. de pesado binaria:
[[1.         0.13608276 0.33333333 0.16666667 0.46291005 0.3086067
  0.27216553 0.18257419 0.28867513 0.16666667]
 [0.13608276 1.         0.13608276 0.27216553 0.12598816 0.
  0.11111111 0.         0.11785113 0.27216553]
 [0.33333333 0.13608276 1.         0.16666667 0.3086067  0.15430335
  0.13608276 0.         0.14433757 0.33333333]
 [0.16666667 0.27216553 0.16666667 1.         0.15430335 0.
  0.13608276 0.         0.14433757 0.16666667]
 [0.46291005 0.12598816 0.3086067  0.15430335 1.         0.28571429
  0.25197632 0.3380617  0.26726124 0.15430335]
 [0.3086067  0.         0.15430335 0.         0.28571429 1.
  0.25197632 0.16903085 0.13363062 0.        ]
 [0.27216553 0.11111111 0.13608276 0.13608276 0.25197632 0.25197632
  1.         0.1490712  0.11785113 0.13608276]
 [0.18257419 0.         0.         0.         0.3380617  0.16903085
  0.1490712  1.         0.15811388 0.        ]
 [0.28867513 0.11785113 0.14433757 0.14433757 0.26726124 0.13363062
  0.11785

---
- **Ejercicio 6:** Utilizando el corpus movie_reviews de la librería NLTK, que es un corpus etiquetado con opiniones sobre películas, se pide utilizar el clasificador MultinomialNB para comprobar qué tipo de representación vectorial puede funcionar mejor para este problema. Hay que dividir el corpus en un 80% para entrenamiento y el 20% restante para test, de forma que para cada prueba que se haga hay que calcular el accuracy y ver cuál va mejor.  
    - Representar con BoW y pesado binario.
    - Representar con BoW y pesado TF.
    - Representar con BoW y pesado TF-IDF.
    - Lo mismo que en los casos anteriores, pero haciendo diferentes preprocesamientos:
        - En el preprocesamiento de los textos, eliminar las stopwords. Probar a obtener de nuevo el accuracy con los tres pesados anteriores.
        - Añadir a la eliminación de stopwords, la lematización, combinado con los tres pesados. 

In [8]:
# Obtener datos
data = []
for categ in movie_reviews.categories():
    for fileid in movie_reviews.fileids(categ):
        text = movie_reviews.open(fileid).read()
        data.append((text, categ))

df = pd.DataFrame(data, columns=("text", "categ"))
X = df["text"]
y = df["categ"]

# Dividir conjunto de datos en entrenamiento (80%) y test (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)
print(f"Tam. de X_train: {X_train.shape}")
print(f"Tam. de X_test: {X_test.shape}")

Tam. de X_train: (1600,)
Tam. de X_test: (400,)


In [9]:
# Repre. con BoW y pesado binario
vectorizer = CountVectorizer(binary=True)
bow_encoded_train = vectorizer.fit_transform(X_train)
bow_encoded_test = vectorizer.transform(X_test)

classifier = MultinomialNB()
classifier.fit(bow_encoded_train.toarray(), y_train)
y_pred = classifier.predict(bow_encoded_test)
print(f"Accuracy con repre. con BoW y pesado binario: {accuracy_score(y_test, y_pred)}")

Accuracy con repre. con BoW y pesado binario: 0.81


In [10]:
# Repre. con BoW y pesado TF
vectorizer = CountVectorizer()
bow_encoded_train = vectorizer.fit_transform(X_train)
bow_encoded_test = vectorizer.transform(X_test)

classifier = MultinomialNB()
classifier.fit(bow_encoded_train.toarray(), y_train)
y_pred = classifier.predict(bow_encoded_test)
print(f"Accuracy con repre. con BoW y pesado TF: {accuracy_score(y_test, y_pred)}")

Accuracy con repre. con BoW y pesado TF: 0.7925


In [11]:
# Repre. con BoW y pesado TF-IDF
vectorizer = TfidfVectorizer()
bow_encoded_train = vectorizer.fit_transform(X_train)
bow_encoded_test = vectorizer.transform(X_test)

classifier = MultinomialNB()
classifier.fit(bow_encoded_train.toarray(), y_train)
y_pred = classifier.predict(bow_encoded_test)
print(f"Accuracy con repre. con BoW y pesado binario: {accuracy_score(y_test, y_pred)}")

Accuracy con repre. con BoW y pesado binario: 0.7975


**Preprocesamiento del texto: eliminar las stopwords.**

In [12]:
def preprocess_sw(text):
    word_tokenized = regexp_tokenize(text, r"[\w|\d]+")
    word_tokenized = [word for word in word_tokenized if word.lower() not in english_sw]
    return " ".join(word_tokenized)

# Aplicamos el preprocesamiento al DF
X_train_copy = X_train.copy()
X_test_copy = X_test.copy()
X_train_preprocessed_sw = X_train_copy.apply(preprocess_sw)
X_test_preprocessed_sw = X_test_copy.apply(preprocess_sw)

In [13]:
# Repre. con BoW y pesado binario (Con eliminación de stopwords)
vectorizer = CountVectorizer(binary=True)
bow_encoded_train = vectorizer.fit_transform(X_train_preprocessed_sw)
bow_encoded_test = vectorizer.transform(X_test_preprocessed_sw)

classifier = MultinomialNB()
classifier.fit(bow_encoded_train.toarray(), y_train)
y_pred = classifier.predict(bow_encoded_test)
print(f"Accuracy con repre. con BoW y pesado binario (sin stopwords): {accuracy_score(y_test, y_pred)}")

Accuracy con repre. con BoW y pesado binario (sin stopwords): 0.8275


In [14]:
# Repre. con BoW y pesado TF (Con eliminación de stopwords)
vectorizer = CountVectorizer()
bow_encoded_train = vectorizer.fit_transform(X_train_preprocessed_sw)
bow_encoded_test = vectorizer.transform(X_test_preprocessed_sw)

classifier = MultinomialNB()
classifier.fit(bow_encoded_train.toarray(), y_train)
y_pred = classifier.predict(bow_encoded_test)
print(f"Accuracy con repre. con BoW y pesado TF (sin stopwords): {accuracy_score(y_test, y_pred)}")

Accuracy con repre. con BoW y pesado TF (sin stopwords): 0.805


In [15]:
# Repre. con BoW y pesado TF-IDF (Con eliminación de stopwords)
vectorizer = TfidfVectorizer()
bow_encoded_train = vectorizer.fit_transform(X_train_preprocessed_sw)
bow_encoded_test = vectorizer.transform(X_test_preprocessed_sw)

classifier = MultinomialNB()
classifier.fit(bow_encoded_train.toarray(), y_train)
y_pred = classifier.predict(bow_encoded_test)
print(f"Accuracy con repre. con BoW y pesado binario (sin stopwords): {accuracy_score(y_test, y_pred)}")

Accuracy con repre. con BoW y pesado binario (sin stopwords): 0.8025


**Preprocesamiento de texto: eliminación de stopwords + lematización.**

In [16]:
# Función auxiliar para obtener los tags de WordNet a partir de los de NLTK
def get_wn_tag(nltk_tag):
    if nltk_tag.startswith("N"):
        return wn.NOUN
    elif nltk_tag.startswith("V"):
        return wn.VERB
    elif nltk_tag.startswith("J"):
        return wn.ADJ
    elif nltk_tag.startswith("R"):
        return wn.ADV
    else:
        return None

# Preprocesar texto con eliminación de stopwords y lematización
def preprocess_sw_lemmatize(text):
    word_tokenized = regexp_tokenize(text, r"\w+")
    word_tokenized = [word for word in word_tokenized if word.lower() not in english_sw]
    tagged_tokens = nltk.pos_tag(word_tokenized)
    lemmatized_words = [WordNetLemmatizer().lemmatize(word, get_wn_tag(tag)) for word, tag in tagged_tokens if get_wn_tag(tag) is not None]
    return " ".join(lemmatized_words)

# Aplicamos el preprocesamiento al DF
X_train_copy = X_train.copy()
X_test_copy = X_test.copy()
X_train_preprocessed_sw_lemmatized = X_train_copy.apply(preprocess_sw_lemmatize)
X_test_preprocessed_sw_lemmatized = X_test_copy.apply(preprocess_sw_lemmatize)

In [17]:
# Repre. con BoW y pesado binario (Con eliminación de stopwords y lematización)
vectorizer = CountVectorizer(binary=True)
bow_encoded_train = vectorizer.fit_transform(X_train_preprocessed_sw_lemmatized)
bow_encoded_test = vectorizer.transform(X_test_preprocessed_sw_lemmatized)

classifier = MultinomialNB()
classifier.fit(bow_encoded_train.toarray(), y_train)
y_pred = classifier.predict(bow_encoded_test)
print(f"Accuracy con repre. con BoW y pesado binario (sin stopwords): {accuracy_score(y_test, y_pred)}")

Accuracy con repre. con BoW y pesado binario (sin stopwords): 0.8125


In [18]:
# Repre. con BoW y pesado TF (Con eliminación de stopwords y lematización)
vectorizer = CountVectorizer()
bow_encoded_train = vectorizer.fit_transform(X_train_preprocessed_sw_lemmatized)
bow_encoded_test = vectorizer.transform(X_test_preprocessed_sw_lemmatized)

classifier = MultinomialNB()
classifier.fit(bow_encoded_train.toarray(), y_train)
y_pred = classifier.predict(bow_encoded_test)
print(f"Accuracy con repre. con BoW y pesado TF (sin stopwords): {accuracy_score(y_test, y_pred)}")

Accuracy con repre. con BoW y pesado TF (sin stopwords): 0.7925


In [19]:
# Repre. con BoW y pesado TF-IDF (Con eliminación de stopwords y lematización)
vectorizer = TfidfVectorizer()
bow_encoded_train = vectorizer.fit_transform(X_train_preprocessed_sw_lemmatized)
bow_encoded_test = vectorizer.transform(X_test_preprocessed_sw_lemmatized)

classifier = MultinomialNB()
classifier.fit(bow_encoded_train.toarray(), y_train)
y_pred = classifier.predict(bow_encoded_test)
print(f"Accuracy con repre. con BoW y pesado binario (sin stopwords): {accuracy_score(y_test, y_pred)}")

Accuracy con repre. con BoW y pesado binario (sin stopwords): 0.7875
